In [1]:
from hiresprv.auth import login
from hiresprv.archive import Archive
from hiresprv.database import Database
import pandas as pd
from hiresprv.idldriver import Idldriver
from hiresprv.download import Download
import numpy as np
import os 

In [2]:
idl = Idldriver('prv.cookies')

In [3]:
filesFromHIRES = os.listdir(path='output/')

In [6]:
emptyFileList = []
nonemptyFileList = []
for fileName in filesFromHIRES[1:]:
    with open('output/'+fileName, 'r') as file:
        for line in file:
            if line[0]=='{':
                emptyFileList.append(fileName)
            else:
                nonemptyFileList.append(fileName)
            break
#     tempHIRESdata = np.genfromtxt("HIRESrvDATA/vstBD+262091.csv",header = True,)[0]
#     if np.isnan(tempHIRESdata):
#         emptyFileList.append(fileName)
#     else:
#         nonemptyFileList.append(fileName)
        

In [7]:
str(round(len(emptyFileList)/len(filesFromHIRES)*100 )) + "% of the Stars do not have HIRES data"

'29% of the Stars do not have HIRES data'

In [8]:
failureStars = [] 
for i in range(len(emptyFileList)):
    failureStars.append(emptyFileList[i].split("vst")[1].split(".csv")[0])

In [9]:
failureStars

['101472',
 '101675',
 '103417',
 '103459',
 '103890',
 '105279',
 '107181',
 '107211',
 '108351',
 '108863',
 '109409',
 '109718',
 '10995',
 '110743',
 '111814',
 '112415',
 '115404A',
 '115617',
 '11964A',
 '124257A',
 '124257B',
 '12974',
 '131156',
 '133295',
 '13357',
 '134353',
 '13483',
 '135101A',
 '135599',
 '136274',
 '13836',
 '140538A',
 '14214',
 '142626',
 '143761',
 '145958A',
 '145958B',
 '146050',
 '146233',
 '146362B',
 '14651',
 '147750',
 '155456',
 '161479',
 '164330',
 '165173',
 '165341A',
 '16760',
 '168009',
 '168603',
 '17354',
 '176733',
 '178911B',
 '182189',
 '182619',
 '183263',
 '18436A',
 '18436B',
 '18702',
 '18916',
 '190360',
 '190412',
 '191359',
 '19618',
 '196201',
 '196761',
 '196885',
 '20155',
 '207485',
 '207992',
 '208880',
 '210302',
 '215274',
 '217107',
 '219834B',
 '224619',
 '224693',
 '233641',
 '238069',
 '24496',
 '27732',
 '28099',
 '2816',
 '28185',
 '32850',
 '33334',
 '33822',
 '34887',
 '35112',
 '36130',
 '41700',
 '49674',
 '51

In [5]:
filesFromHIRES

['.ipynb_checkpoints',
 'vst10008.csv',
 'vst10011.csv',
 'vst10013.csv',
 'vst10015.csv',
 'vst100180.csv',
 'vst100337.csv',
 'vst100623.csv',
 'vst10145.csv',
 'vst101472.csv',
 'vst101501.csv',
 'vst101675.csv',
 'vst101847.csv',
 'vst101904.csv',
 'vst101959.csv',
 'vst102071.csv',
 'vst10212.csv',
 'vst102195.csv',
 'vst102365.csv',
 'vst102444.csv',
 'vst102956.csv',
 'vst103047.csv',
 'vst103417.csv',
 'vst103459.csv',
 'vst103616.csv',
 'vst103813.csv',
 'vst103828.csv',
 'vst10383.csv',
 'vst103847.csv',
 'vst103890.csv',
 'vst104263.csv',
 'vst104304.csv',
 'vst104389.csv',
 'vst10442.csv',
 'vst104437.csv',
 'vst104588.csv',
 'vst10476.csv',
 'vst10479.csv',
 'vst104800.csv',
 'vst104860.csv',
 'vst105.csv',
 'vst105279.csv',
 'vst105631.csv',
 'vst105811.csv',
 'vst106088.csv',
 'vst106270.csv',
 'vst106279.csv',
 'vst106314.csv',
 'vst106421.csv',
 'vst10697.csv',
 'vst10700.csv',
 'vst107146.csv',
 'vst107148.csv',
 'vst107181.csv',
 'vst107211.csv',
 'vst10780.csv',
 'v

In [8]:
#HIRES 
login('prv.cookies')

state = Database('prv.cookies')

KOA userid: jgussman
KOA Password: ········
Successful login as jgussman


In [14]:
oneOrnoneRVdata = []
nonemptyDataList = []
for TargetName in failureStars:
    star_info = state.target_info(TargetName)

    df = pd.read_html(star_info, header=0)[0]
    df = df.drop(df[(df['OBTYPE'] != 'RV\xa0observation')].index)
    if len(df) <= 1:
        oneOrnoneRVdata += [TargetName]
        
    else:
        nonemptyDataList += [TargetName]

In [15]:
len(oneOrnoneRVdata)

159

In [19]:
bigq= ""
for TargetName in failureStars:
    star_info = state.target_info(TargetName)

    df = pd.read_html(star_info, header=0)[0]
    #Drop all the rows that are NOT RV observations 
    df = df.drop(df[(df['OBTYPE'] != 'RV\xa0observation')].index)
    if not df.empty: #If the star has no RV observations. It probably just has a template
        try:
            filename = df['FILENAME'][0]
            filename = filename.replace("r","")
            filename = filename.split(".")[0]
        except: 
            filename = df['FILENAME'].array[0]
            filename = filename.replace("r","")
            filename = filename.split(".")[0]
        #Create the Querry to Extract the RV Data 
        querry_To_NEXsci = "template {0} {1}".format(TargetName,filename)

        for RVObv in df.to_numpy():
            querry_To_NEXsci += '\nrv {0} {1}'.format(TargetName,RVObv[3])

        querry_To_NEXsci += "\nrvcurve {0}".format(TargetName)

        
        bigq += querry_To_NEXsci

In [23]:
rtn = idl.run_script(bigq) 

status= ok
msg= Script running in background. Consult monitor for status.


In [25]:
#Download the Data
data = Download('prv.cookies', './')
somethingWentWrongStarList = []
for s in failureStars:
    rtn = data.rvcurve(s)
    try:
        with open('vst{0}.csv'.format(s), 'r') as file:
          for line in file:
            print(line, end='')
    except:
        somethingWentWrongStarList+=[s]

{"status":"Failed to open requested file vstBD 262091.csv in your workspace."}
{"status":"Failed to open requested file vstBD 312447.csv in your workspace."}
{"status":"Failed to open requested file vstBD 374783.csv in your workspace."}
{"status":"Failed to open requested file vstBD 374784.csv in your workspace."}
{"status":"Failed to open requested file vstBD-103166.csv in your workspace."}
{"status":"Failed to open requested file vstCK00005.csv in your workspace."}
{"status":"Failed to open requested file vstCK00007.csv in your workspace."}
{"status":"Failed to open requested file vstCK00041.csv in your workspace."}
{"status":"Failed to open requested file vstCK00063.csv in your workspace."}
{"status":"Failed to open requested file vstCK00069.csv in your workspace."}
{"status":"Failed to open requested file vstCK00070.csv in your workspace."}
{"status":"Failed to open requested file vstCK00072.csv in your workspace."}
{"status":"Failed to open requested file vstCK00075.csv in your wo

{"status":"Failed to open requested file vst140538A.csv in your workspace."}
{"status":"Failed to open requested file vst14214.csv in your workspace."}
{"status":"Failed to open requested file vst142626.csv in your workspace."}
{"status":"Failed to open requested file vst143761.csv in your workspace."}
{"status":"Failed to open requested file vst145958A.csv in your workspace."}
{"status":"Failed to open requested file vst145958B.csv in your workspace."}
{"status":"Failed to open requested file vst146050.csv in your workspace."}
{"status":"Failed to open requested file vst146233.csv in your workspace."}
{"status":"Failed to open requested file vst146362B.csv in your workspace."}
{"status":"Failed to open requested file vst14651.csv in your workspace."}
{"status":"Failed to open requested file vst147750.csv in your workspace."}
{"status":"Failed to open requested file vst155456.csv in your workspace."}
{"status":"Failed to open requested file vst161198.csv in your workspace."}
{"status":

{"status":"Failed to open requested file vst70516.csv in your workspace."}
{"status":"Failed to open requested file vst70843.csv in your workspace."}
{"status":"Failed to open requested file vst72760.csv in your workspace."}
{"status":"Failed to open requested file vst73534.csv in your workspace."}
{"status":"Failed to open requested file vst78366.csv in your workspace."}
{"status":"Failed to open requested file vst78558.csv in your workspace."}
{"status":"Failed to open requested file vst79498.csv in your workspace."}
{"status":"Failed to open requested file vst82074.csv in your workspace."}
{"status":"Failed to open requested file vst82328.csv in your workspace."}
{"status":"Failed to open requested file vst82443.csv in your workspace."}
{"status":"Failed to open requested file vst82885.csv in your workspace."}
{"status":"Failed to open requested file vst86728.csv in your workspace."}
{"status":"Failed to open requested file vst87001.csv in your workspace."}
{"status":"Failed to open

{"status":"Failed to open requested file vstHTR211-001.csv in your workspace."}
{"status":"Failed to open requested file vstHTR211-003.csv in your workspace."}
{"status":"Failed to open requested file vstHTR212-002.csv in your workspace."}
{"status":"Failed to open requested file vstHTR213-002.csv in your workspace."}
{"status":"Failed to open requested file vstHTR213-008.csv in your workspace."}
{"status":"Failed to open requested file vstHTR239-004.csv in your workspace."}
{"status":"Failed to open requested file vstHTR239-008.csv in your workspace."}
{"status":"Failed to open requested file vstHTR240-003.csv in your workspace."}
{"status":"Failed to open requested file vstHTR241-007.csv in your workspace."}
{"status":"Failed to open requested file vstHTR248-002.csv in your workspace."}
{"status":"Failed to open requested file vstHTR258-002.csv in your workspace."}
{"status":"Failed to open requested file vstHTR263-005.csv in your workspace."}
{"status":"Failed to open requested file

{"status":"Failed to open requested file vstXO-3.csv in your workspace."}
{"status":"Failed to open requested file vstXO-4.csv in your workspace."}
